In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "C:\Users\Tushar\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


if you want to see logging events.


From Strings to Vectors
This time, let’s start from documents represented as strings:

In [6]:
from gensim import corpora

C:\Users\Tushar\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-11-28 18:46:39,195 : INFO : 'pattern' package not found; tag filters are not available for English


In [7]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [10]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())

texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)        

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [11]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)

2017-11-28 19:17:58,248 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-11-28 19:17:59,018 : INFO : built Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...) from 9 documents (total 29 corpus positions)
2017-11-28 19:17:59,563 : INFO : saving Dictionary object under C:\Users\Tushar\AppData\Local\Temp\deerwester.dict, separately None
2017-11-28 19:18:00,956 : INFO : saved C:\Users\Tushar\AppData\Local\Temp\deerwester.dict


Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)


In [12]:
print(dictionary.token2id)

{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'user': 4, 'system': 5, 'response': 6, 'time': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [14]:
new_doc = "Human machine interface for lab abc computer applications"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1), (2, 1)]


In [15]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use
for c in corpus:
    print(c)

2017-11-28 19:36:51,057 : INFO : storing corpus in Matrix Market format to C:\Users\Tushar\AppData\Local\Temp\deerwester.mm
2017-11-28 19:36:51,245 : INFO : saving sparse matrix to C:\Users\Tushar\AppData\Local\Temp\deerwester.mm
2017-11-28 19:36:51,270 : INFO : PROGRESS: saving document #0
2017-11-28 19:36:51,376 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-11-28 19:36:51,387 : INFO : saving MmCorpus index to C:\Users\Tushar\AppData\Local\Temp\deerwester.mm.index


[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (4, 1), (5, 1), (8, 1)]
[(0, 1), (5, 2), (8, 1)]
[(4, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(3, 1), (10, 1), (11, 1)]


In [15]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use
for c in corpus:
    print(c)

2017-11-28 19:36:51,057 : INFO : storing corpus in Matrix Market format to C:\Users\Tushar\AppData\Local\Temp\deerwester.mm
2017-11-28 19:36:51,245 : INFO : saving sparse matrix to C:\Users\Tushar\AppData\Local\Temp\deerwester.mm
2017-11-28 19:36:51,270 : INFO : PROGRESS: saving document #0
2017-11-28 19:36:51,376 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-11-28 19:36:51,387 : INFO : saving MmCorpus index to C:\Users\Tushar\AppData\Local\Temp\deerwester.mm.index


[(0, 1), (1, 1), (2, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (4, 1), (5, 1), (8, 1)]
[(0, 1), (5, 2), (8, 1)]
[(4, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(3, 1), (10, 1), (11, 1)]


In [5]:
class MyCorpus(object):
    def __iter__(self):
        for line = line in open('datasets/mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [3]:
corpus_memory_friendly = MyCorpus() # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [ ]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)